I will be examining which boroughs are the best to open a new fast food restaurant in Toronto. I assume that areas with the lowest amount of (fast food) restaurants would be the best ones.

Using Foursquare location data, I will find out which boroughs have the lowest number of fast food restaurants, and also where exactly in these boroughs are restaurants of other kinds located.

Let's start by importing relevant libraries:


In [14]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium 
import folium # map rendering library

print('Libraries imported.')


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --yes
Libraries imported.


Let's preprocess the dataframe with geospatial data of Toronto like in Week 3 Assignment:

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
len(df_list)

3

In [17]:
df = df_list[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [18]:
#drop borough not assigned

indexBorough = df[df['Borough'] == 'Not assigned'].index
df.drop(indexBorough , inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
# Not assigned  neighborhood same as borough

df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [20]:
# combine rows with same postal codes

df.groupby(['Postal Code','Neighbourhood'], sort = False).agg(lambda x: ','.join(x))
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [21]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [22]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv', sep=",")
# geo_df.columns = geo_df.columns.str.strip()
geo_df.loc[geo_df['Postal Code'] == 'M4A']

,Postal Code,Latitude,Longitude
34,M4A,43.725882,-79.315572


In [23]:
#merge the two dataframes on their matching postal codes 

df_merged = pd.merge(df,geo_df, on=['Postal Code'])
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [24]:
#Use geopy library to get the latitude and longitude values of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let's create a map of Toronto with all boroughs marked:

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version:

In [26]:
CLIENT_ID = 'TBZYDSUX2RQ5RBGFWHYWOC40PHAQZ4QBZPUQ0VHVQ3JJVLYN' # your Foursquare ID
CLIENT_SECRET = 'QC554ZZD3LPHRRX3IGQNHCWWC5040LC0K3INTC1SI2F0VHFG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TBZYDSUX2RQ5RBGFWHYWOC40PHAQZ4QBZPUQ0VHVQ3JJVLYN
CLIENT_SECRET:QC554ZZD3LPHRRX3IGQNHCWWC5040LC0K3INTC1SI2F0VHFG


Let's see how many fast food restaurants are in each neighborhood:

In [27]:
#create a function to retrieve a list of venues in all neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
#run the above function on each neighborhood and create a new dataframe called all_venues

all_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Analyze each neighborhood:

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = all_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's create a new dataframe that shows the number of fast food restaurants only:

In [59]:
t_food = toronto_onehot[['Neighbourhood','Fast Food Restaurant']]

# rename columns with spaces and replace spaces with "_"
t_food.columns = [c.replace(' ', '_') for c in t_food.columns]

#dropping 2nd column rows with null values 
t_food = t_food[t_food.Fast_Food_Restaurant != 0]
t_food

,Neighbourhood,Fast_Food_Restaurant
98,"Malvern, Rouge",1
141,"Garden District, Ryerson",1
150,"Garden District, Ryerson",1
539,Hillcrest Village,1
578,Thorncliffe Park,1
615,"Richmond, Adelaide, King",1
732,"Fairview, Henry Farm, Oriole",1
734,"Fairview, Henry Farm, Oriole",1
741,"Fairview, Henry Farm, Oriole",1
763,"Fairview, Henry Farm, Oriole",1


In [56]:
t_food.shape

(28, 2)

We can see that 28 neighborhoods have fast food restaurants already and would therefore not be the best choice to open a new one. As some of the neighborhoods in the original dataframe are listed multiple times (just with different postal codes), let's take a look at neighborhoods that do not have a fast food restaurant and at the same time their name is not the same like the 28 neighborhoods above.
---



In [61]:
no_food = toronto_onehot[['Neighbourhood','Fast Food Restaurant']]

(2130, 2)

In [65]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Garden District, Ryerson']
no_food.shape

(2030, 2)

In [66]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Fairview, Henry Farm, Oriole']
no_food.shape

(1964, 2)

In [67]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Malvern, Rouge']
no_food.shape

(1963, 2)

In [68]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Hillcrest Village']
no_food.shape

(1958, 2)

In [69]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Thorncliffe Park']
no_food.shape

(1938, 2)

In [70]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Richmond, Adelaide, King']
no_food.shape

(1841, 2)

In [71]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Toronto Dominion Centre, Design Exchange']
no_food.shape

(1741, 2)

In [72]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'India Bazaar, The Beaches West']
no_food.shape

(1723, 2)

In [73]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Commerce Court, Victoria Hotel']
no_food.shape

(1623, 2)

In [74]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'Willowdale, Willowdale East']
no_food.shape

(1588, 2)

In [75]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'High Park, The Junction South']
no_food.shape

(1564, 2)

In [77]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != 'North Toronto West, Lawrence Park']
no_food.shape

(1545, 2)

In [82]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Clarks Corners, Tam O'Shanter, Sullivan"]
no_food.shape

(1534, 2)

In [83]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "New Toronto, Mimico South, Humber Bay Shores"]
no_food.shape

(1521, 2)

In [84]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Steeles West, L'Amoreaux West"]
no_food.shape

(1509, 2)

In [85]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Stn A PO Boxes"]
no_food.shape

(1410, 2)

In [86]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "First Canadian Place, Underground city"]
no_food.shape

(1310, 2)

In [87]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Church and Wellesley"]
no_food.shape

(1231, 2)

In [88]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens"]
no_food.shape

(1223, 2)

In [89]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Business reply mail Processing Centre, South Central Letter Processing Plant Toronto"]
no_food.shape

(1206, 2)

In [90]:
#dropping first column rows with values identical with the t_food dataframe
no_food = no_food[no_food.Neighbourhood != "Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West"]
no_food.shape

(1191, 2)

In [92]:
no_food.head()

,Neighbourhood,Fast Food Restaurant
0,Parkwoods,0
1,Parkwoods,0
2,Victoria Village,0
3,Victoria Village,0
4,Victoria Village,0


We can see that around half of Toronto's neighborhoods might be a good place to open a new fast food restaurants. Let's visualize these neighborhoods on a map:

In [97]:
#add latitude and longitude data to the no_food dataframe

#merge the two dataframes on their matching neighborhoods

df_geo = pd.merge(df_merged,no_food, on=['Neighbourhood'])
df_geo.shape
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Fast Food Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,0
1,M3A,North York,Parkwoods,43.753259,-79.329656,0
2,M4A,North York,Victoria Village,43.725882,-79.315572,0
3,M4A,North York,Victoria Village,43.725882,-79.315572,0
4,M4A,North York,Victoria Village,43.725882,-79.315572,0


In [98]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We can see that neighborhoods with no fast food restaurants are scattered all over Toronto, however, most of them are in the city centre. Let's examine these neighborhoods closer to determine where exactly restaurants are located in them, so that we can choose a specific area with not many other restaurant for the new fast food restaurant. I will examine the following neighborhoods using the Foursquare API:


1.   Harbourfront East, Union Station, Toronto Islands
2.   Regent Park, Harbourfront
3. Central Bay Street
4. University of Toronto, Harbord
5. Kensington Market, Chinatown, Grange Park
6. Berczy Park





In [94]:
CLIENT_ID = 'TBZYDSUX2RQ5RBGFWHYWOC40PHAQZ4QBZPUQ0VHVQ3JJVLYN' # your Foursquare ID
CLIENT_SECRET = 'QC554ZZD3LPHRRX3IGQNHCWWC5040LC0K3INTC1SI2F0VHFG' # your Foursquare Secret
ACCESS_TOKEN = 'CIDMXW12M4ELZEX3PPHN3DFR4GDP3CXI15MG5BXBEKEU0LW1' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TBZYDSUX2RQ5RBGFWHYWOC40PHAQZ4QBZPUQ0VHVQ3JJVLYN
CLIENT_SECRET:QC554ZZD3LPHRRX3IGQNHCWWC5040LC0K3INTC1SI2F0VHFG


In [99]:
#get latitude and longitude of an address in Toronto City Center

address = '522 University Ave, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.65544965 -79.38913923767831


Search for restaurants:

In [100]:
search_query = 'Restaurant'
radius = 2000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [101]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TBZYDSUX2RQ5RBGFWHYWOC40PHAQZ4QBZPUQ0VHVQ3JJVLYN&client_secret=QC554ZZD3LPHRRX3IGQNHCWWC5040LC0K3INTC1SI2F0VHFG&ll=43.65544965,-79.38913923767831&oauth_token=CIDMXW12M4ELZEX3PPHN3DFR4GDP3CXI15MG5BXBEKEU0LW1&v=20180604&query=Restaurant&radius=2000&limit=30'

In [102]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff1127f54a4277f8fedafae'},
 'notifications': [{'item': {'unreadCount': 0}, 'type': 'notificationTray'}],
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'primary': True,
      'shortName': 'American'}],
    'hasPerk': False,
    'id': '4ad4c05ff964a52048f720e3',
    'location': {'address': '110 Chestnut Street',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 265,
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'postalCode': 'M5G 1R3',
     'state': 'ON'},
    'name': 'He

In [104]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1609634431,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",265,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1609634431,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",380,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
2,4ae29812f964a520288f21e3,Swatow Restaurant 汕頭小食家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1609634431,False,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",761,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,NaN
3,4ad4c060f964a5207ff720e3,Rol San Restaurant 龍笙棧,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1609634431,False,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",776,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,NaN
4,4b266f05f964a520657b24e3,Goldstone Noodle Restaurant 金石,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1609634431,False,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",799,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,NaN


In [105]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",265,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",380,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
2,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",761,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,4ae29812f964a520288f21e3
3,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",776,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,4ad4c060f964a5207ff720e3
4,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",799,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,4b266f05f964a520657b24e3
5,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,"[{'label': 'display', 'lat': 43.6531846832903,...",657,NaN,CA,Toronto,ON,Canada,"[476 Dundas St. (Huron), Toronto ON, Canada]",Huron,NaN,4d06c1ccc2e53704428cc567
6,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,"[{'label': 'display', 'lat': 43.65248783033554...",749,M5T 2E3,CA,Toronto,ON,Canada,"[261 Spadina Avenue. Upper level, Toronto ON M...",NaN,NaN,5c33c7cd8194fc002c66f72e
7,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,"[{'label': 'display', 'lat': 43.65533674412141...",786,M5T 2G3,CA,Toronto,ON,Canada,"[353 Spadina Ave., Toronto ON M5T 2G3, Canada]",NaN,NaN,4b074bb1f964a52077fb22e3
8,Ka Chi Korean Restaurant,Korean Restaurant,8 St Andrew St.,43.654307,-79.399277,"[{'label': 'display', 'lat': 43.65430655228665...",826,M5T 1K6,CA,Toronto,ON,Canada,"[8 St Andrew St. (at Spadina Ave.), Toronto ON...",at Spadina Ave.,NaN,4ba2d00cf964a5203e1b38e3
9,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",785,NaN,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,4b072e9df964a52009f922e3


Let's visualize the nearby restaurants:

In [106]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto city center

# add a red circle marker to represent the address in the middle of the 6 neighborhoods choosen before
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

We can see that most restaurants are scattered around Chinatown, followed by the central train station and St. Patrick station. At the same time, there are no restaurants around University-Rosedale. I therefore suggest to open a new fast food restaurant in that area.

